In [86]:
%matplotlib osx

In [87]:
import lop.slowly_changing_regression.hoi_measures.datatools as datatools

In [95]:
datatensor = datatools.data_dir_to_tensor("lop/slowly_changing_regression/data/sgd/bp/relu/0")

In [96]:
datalist = [datatensor[i, :, :] for i in range(datatensor.shape[0])]

In [90]:
datalist[0].shape

torch.Size([81, 110])

In [91]:
import pandas as pd
import seaborn as sns

idx = 10
nparams = 5

plot1 = sns.pairplot(pd.DataFrame(datalist[1][:, idx:idx+nparams].numpy()))

plot1.savefig("pairplot.png")

plot2 = sns.pairplot(pd.DataFrame(datalist[-1][:, idx:idx+nparams].numpy()))

plot2.savefig("pairplot2.png")

In [11]:
from thoi.measures.gaussian_copula import multi_order_measures, nplets_measures
from thoi.heuristics import simulated_annealing, greedy

In [92]:
import numpy as np

covmat = np.cov(datalist[99][:, :].T.numpy())

if np.any(np.abs(covmat) <  1e-4):
    print("Warning: some variables are constant")
    print(np.min(np.abs(covmat)))

sns.heatmap(covmat)


5.8803921930881164e-06


<Axes: >

In [93]:
testmeasure = nplets_measures(datalist[999][:, :39], [range(39)])
testmeasure

tensor([[[2.7302, 1.1876, 1.5426, 3.9178]]], dtype=torch.float64)

In [21]:
npletsids, scores = greedy(datalist[-2][:, :40], largest = False, metric = "o", batch_size=50_000)

In [19]:
scores_cpu = scores.cpu().numpy()

In [22]:
greedy_min = np.min(scores_cpu, axis=0)

import matplotlib.pyplot as plt

plt.plot(greedy_min)

# todo

- Matriz tiene cov 0 en algunas coords y tira NaN. Podríamos sacarlo nomás
- Quizas hay params que no se indeterminan por pertenecer al mismo grupo de parametros.
-
Si se vuelve mucho podemos estudiar los eigvals de la matriz de cov. Si hay uno muy grande nos habla de redundancia.

In [99]:
import numpy as np
from tqdm import trange
# Define the kernel for averaging the nearest 40000 elements

def timebin_data(data, binsize):
    return np.array([np.mean(data[i*binsize:(i+1)*binsize]) for i in range(len(data) // binsize)])

avg_errs = np.zeros(25)
for idx in trange(100):
    datadict = datatools.load_data("lop/slowly_changing_regression/data/sgd/cbp/relu/0/" + str(idx))
    # Perform the convolution
    smoothed_errs = timebin_data(datadict["errs"], 40000)

# Plot the smoothed errors
plt.clf()
plt.plot(np.arange(25) * 40000, smoothed_errs)

100%|██████████| 100/100 [00:10<00:00,  9.52it/s]


In [100]:
avg_errs = np.zeros(25)
for idx in trange(100):
    datadict = datatools.load_data("lop/slowly_changing_regression/data/sgd/bp/relu/0/" + str(idx))
    # Perform the convolution
    smoothed_errs = timebin_data(datadict["errs"], 40000)
plt.plot(np.arange(25) * 40000, smoothed_errs)

100%|██████████| 100/100 [00:10<00:00,  9.91it/s]


In [98]:
import matplotlib.pyplot as plt
import numpy as np
import imageio.v2 as imageio

# Placeholder for storing generated images
frames = []

A = np.arange(16)*60

# Iterate through datalist
for i in A:
    npletsids, scores = greedy(datalist[i][:, :40], largest=False, metric="o", batch_size=50_000)
    scores_cpu = scores.cpu().numpy()
    greedy_min = np.min(scores_cpu, axis=0)
    
    npletsids2, scores2 = greedy(datalist[i][:, :40], largest=True, metric="o", batch_size=50_000)
    scores_cpu2 = scores2.cpu().numpy()
    greedy_max = np.max(scores_cpu2, axis=0)

    # Plot the result
    plt.figure(figsize=(6, 4))
    plt.plot(greedy_max, label="Maximum O-information")
    plt.plot(greedy_min, label="Minimum O-information")
    plt.title(f"Order v/s O-information for training step {i*1000}")
    plt.xlabel("Order")
    plt.ylabel("O-Info")
    plt.legend()
    plt.grid()
    plt.xlim([0,35])
    plt.ylim([-1.0, 1.5])

    # Save the frame as an image in memory
    filename = f"frame.png"
    plt.savefig(filename)
    frames.append(imageio.imread(filename))  # Add the image to the frames list
    plt.close()  # Close the plot to avoid memory issues

# Create a GIF from the frames
output_gif = "bp_greedy_min_visualization.gif"
imageio.mimsave(output_gif, frames, fps=1)  # Adjust fps for speed

print(f"GIF saved as {output_gif}")

GIF saved as bp_greedy_min_visualization.gif
